# Popular Names in South Australia

This project explores popular baby names from 1944-2013 in South Australia. The dataset is taken from https://data.sa.gov.au/data/dataset/popular-baby-names

### Load in the data
There are multiple files, two per year - one for female names and one for male names.<br>
We combine all the files into one dataframe.
We also add new `gender` and `year` columns.

In [1]:
import glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

file_list = glob.glob('./data/*.csv') 
df = pd.DataFrame()

for f in file_list:
    df_tmp = pd.read_csv(f)
    i = f.find('cy')
    df_tmp['year'] = f[i+2:i+6]
    i = f.find('\\')
    df_tmp['sex'] = f[i+1]
    df = pd.concat([df, df_tmp],ignore_index=True)

In [2]:
df.head()

,Given Name,Amount,Position,year,sex
0,MARGARET,341,1,1944,f
1,HELEN,209,2,1944,f
2,JUDITH,189,3,1944,f
3,PATRICIA,186,4,1944,f
4,PAMELA,160,5,1944,f


The `position` column is not needed so we drop it.

In [3]:
df.drop('Position', axis=1, inplace=True)

In [4]:
df.head()

,Given Name,Amount,year,sex
0,MARGARET,341,1944,f
1,HELEN,209,1944,f
2,JUDITH,189,1944,f
3,PATRICIA,186,1944,f
4,PAMELA,160,1944,f


### Clean the data
First we identify duplicated rows on `Given Name` and `year` and sum the amount, results stored in a series.

In [5]:
dups = df[df.duplicated(['Given Name','year'],keep=False)].groupby(['Given Name','year']).Amount.sum()

Convert the series into a new dataframe.

In [6]:
df_dup = dups.to_frame()

In [7]:
df_dup.head()

Amount
Given Name year        
(NOT       1968       8
AARON      1963       8
           1966       7
           1968      21
           1969      10

In [8]:
df.shape[0]

210939

In [9]:
df.duplicated(['Given Name','year'],keep=False).value_counts()

False    193224
True      17715
dtype: int64

We drop duplicates in our original dataframe, keeping only one occurrence.

In [10]:
#drop duplicates keep first occurrence only
df.drop_duplicates(['Given Name','year'],inplace=True) 

In [11]:
df.duplicated(['Given Name','year'],keep=False).value_counts()

False    201935
dtype: int64

Convert index into columns so we can easily merge.

In [12]:
df_dup.reset_index(level=['Given Name', 'year'], inplace=True) #convert index into columns
df_dup.head()

,Given Name,year,Amount
0,(NOT,1968,8
1,AARON,1963,8
2,AARON,1966,7
3,AARON,1968,21
4,AARON,1969,10


Merge the new dataframe `Amount` value into our original dataframe using `Given Name` and `year` as the key.<br>
This should give us the sum amount in a new column.

In [13]:
df2 = pd.merge(df, df_dup, on=['Given Name','year'], how='left') #like a SQL left join

In [14]:
df2.head()

,Given Name,Amount_x,year,sex,Amount_y
0,MARGARET,341,1944,f,343.0
1,HELEN,209,1944,f,NaN
2,JUDITH,189,1944,f,NaN
3,PATRICIA,186,1944,f,NaN
4,PAMELA,160,1944,f,NaN


In [15]:
df2.shape[0]

201935

We are going to use `Amount_y` as our amount column, so we fill all the NaN with the value from the `Amount_x` column.

In [16]:
#this is like an if then assign to specific column
df2.loc[df2.Amount_y.isnull(), 'Amount_y'] = df2['Amount_x']

In [17]:
df2.head()

,Given Name,Amount_x,year,sex,Amount_y
0,MARGARET,341,1944,f,343.0
1,HELEN,209,1944,f,209.0
2,JUDITH,189,1944,f,189.0
3,PATRICIA,186,1944,f,186.0
4,PAMELA,160,1944,f,160.0


Drop the `Amount_x` column as we don't need it anymore and change the type of `Amount_y` to an integer.<br>
We also need to convert `year` to integer as we will use it for both our labels and our index values.

In [18]:
df2.drop('Amount_x', axis=1, inplace=True)

In [28]:
df2.Amount_y = df2.Amount_y.astype(int)
df2.year = df2.year.astype(int)
df2.head()

,Given Name,year,sex,Amount_y
0,MARGARET,1944,f,343
1,HELEN,1944,f,209
2,JUDITH,1944,f,189
3,PATRICIA,1944,f,186
4,PAMELA,1944,f,160


### Plot the graph
Get a list of all unique years in `year_label`. This will be used for our x-axis labels and index values.

In [37]:
all_years = df2.year.unique()
#year_index = np.arange(all_years.min(),all_years.max()+1)

Prepare the data for plotting<br>
Change the names to the name you would like to explore.<br>
`max_each_year` will be used to display the most popular name for that year

In [ ]:
#plot takes the index as the x value and uses any numeric column as the y
name1 = df2.loc[df2['Given Name']=='ANDREW'].sort_values('year')
name2 = df2.loc[df2['Given Name']=='MICHAEL'].sort_values('year')
max_each_year = df2.loc[df2.groupby('year').Amount_y.idxmax()]
max_each_year.reset_index(drop=True, inplace=True) #reset the index

Plot the graph

In [ ]:
fig=plt.figure(figsize=(16,8))
ax=fig.add_subplot(1,1,1)

ax.plot(name1.year, name1.Amount_y)
ax.plot(name2.year, name2.Amount_y)
ax.plot(max_each_year.year, max_each_year.Amount_y, '.-')

ax.set_xticks(all_years)
ax.set_xticklabels(all_years, rotation='vertical')
ax.set_xlabel('Year')
ax.set_ylabel('Amount')
ax.set_title('Baby Names (South Australia)')

for i, row in max_each_year.iterrows():
    if i%2: #plot every other name because graph is too crowded
        ax.text(row.year,row.Amount_y,row['Given Name'])

ax.legend(labels=(name1.iloc[0,0], name2.iloc[0,0],'Most popular name for the year'))